<a href="https://colab.research.google.com/github/macnack/Hulanki/blob/main/01_Przeszukiwanie_i_planowanie_%C5%9Bcie%C5%BCki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorium Wstęp do Sztucznej Inteligencji

## Ćwiczenie 1. Przeszukiwanie i planowanie ścieżki

<img src="https://drive.google.com/uc?id=12-CL76-6gDAkulaB5AZA6u23iCnxSeDQ" width="500"/>


Politechnika Poznańska

Instytut Robotyki i Inteligencji Maszynowej

Joanna Piasek, Jan Wietrzykowski


# Grid world w 2-D

Jednym z najczęściej występujących problemów w robotyce mobilnej jest planowanie ścieżki. Robot lub, ogólniej, autonomiczny agent musi zaplanować ścieżkę od swojej aktualnej pozycji do pozycji celu, np. szafki z ciasteczkami, które ma dostarczyć. Jeśli rozpatrywany obszar poszukiwań nie jest duży, zazwyczaj można go przedstawić za pomocą regularnej siatki:

<img src="https://drive.google.com/uc?id=1JYMV3316HAsV0BIgBZhh-4GNZ7o0xvUZ" width="500"/>

W przybiliżeniu tym przyjmujemy, że agent może znajdować się w tylko jednej z komórek. W ten sposób otrzymujemy zdyskretyzowaną listę stanów agenta, ponieważ komórka, w której się znajduje, jest jednocześnie jego stanem. Jeśli oznaczymy komórki przez ich współrzędne, stan możemy zapisać jako `(x, y)`, jak na rysunku poniżej:

<img src="https://drive.google.com/uc?id=1JDUiUdrllC0F11hiANzA9xa_v8JdRZlV" width="180"/>

Większość algorytmów planowania ścieżki używa grafów do reprezentacji problemu, więc i w naszym przypadku przedstawimy go w postaci grafu. Wierzchołkami grafu będą stany agenta, czyli lokacje (komórki), w których może znaleźć się agent. Jeśli agent może poruszać się we wszystkich kierunkach, z danego stanu możemy przemieścić się do wszystkich sąsiadujących stanów. W przykładzie z rysunku poniżej, ze stanu `(3, 2)` możemy przejść do stanów `(2, 2)`, `(3, 3)` oraz `(4, 2)`, co zostało zilustrowane wycinkiem grafu wrysowanym w mapę.

<img src="https://drive.google.com/uc?id=1Z-O6IaqhsJd2PVHI7akCPE1Y6g6fOs2r" width="180"/>




#Sposoby reprezentacji grafów 

Istnieją różne sposoby reprezentacji grafów. Przypomnijmy sobie dwa najbardziej użyteczne.

**Macierz sąsiedztwa**

Graf o $n$ wierzchołkach możemy reprezentować za pomocą tablicy A o rozmiarze n na n elementów. Rzędy i kolumny macierzy reprezentują wierzchołki: $A[i][j] = 1$ oznacza, że istnieje krawędź pomiędzy wierzchołkami $i$ oraz $j$, z kolei $A[i][j] =0$ wskazuje, ze podana krawędź nie istnieje. W grafach z wagami, wagę krawędzi zapisuje się w miejscu 1.  

Zaletą takiego sposobu reprezentacji grafu jest możliwość natychmiastowego sprawdzenia istnienia krawędzi między dwoma wierzchołkami. Podobnie szybko możemy dodawać lub usuwać dowolne krawędzie. Wadą tej metody zapisu jest jej złożoność pamięciowa - wymaga użycia tablicy o rozmiarze $n^2$. W przypadku grafów rzadkich (tj. takich, w których jest mało krawędzi w stosunku do liczby wierzchołków), większość pamięci przechowuje bezużyteczne informacje.


**Listy sąsiedztwa**

Grafy rzadkie można reprezentować za pomocą list sąsiedztwa. Wówczas dla każdego wierzchołka przechowujemy wyłącznie informacje o wierzchołkach będących jego sąsiadami. Zmniejsza to złożoność pamięciową dla grafu złożonego z $n$ wierzchołków i $k$ krawędzi do $O(n+k)$. Jednakże, aby stwierdzić występowanie krawędzi między wierzchołkami $i$ oraz $j$, musimy przeszukać całą listę sąsiadów wierzchołka $i$ lub $j$.

<img src="https://drive.google.com/uc?id=1bpv6P1SiyFTG-_oo8QExOFgKfxcY8DF0"/>


# Breadth-First Search (BFS)

Za pomocą przeszukiwania wszerz (ang. *Breadth-First Search, BFS*) możliwe jest znalezienie najkrótszej ścieżki w grafie, którego wszystkie krawędzie mają jednakową długość (najczęściej przyjmuje się długość równą 1). Algorytm ten polega na odwiedzaniu wierzchołków w kolejności rosnących odległości od wierzchołka startowego. W ten sposób, po dotarciu do wierzchołka docelowego, mamy gwarancję znalezienia najkrótszej ścieżki. Dokładny opis algorytmu można znaleźć [tutaj](http://www.algorytm.edu.pl/grafy/bfs.html).

Jeśli po dotarciu do wierzchołka docelowego chcemy odtworzyć znalezioną ścieżkę, konieczne jest zapisywanie kolejnych kroków. Można do tego celu wykorzystać słownik, który dla każdego wierzchołka dodanego do kolejki będzie przechowywał wierzchołek, z którego do niego dotarto. Za pomocą pseudokodu algorytm ten można przedstawić następująco:

```python

# s - wierzchołek startowy
# g - wierzchołek docelowy
# nodes - lista wierzchołków

# lista odwiedzonych wierzchołków
visited = set()
# słownik poprzedników
parent = {n: None for n in nodes}

q = Queue()

# dodaj wierzchołek startowy
push(q, s)
# ustaw jego poprzednika jako jego samego, aby oznaczyć go jako odwiedzony
parent[s] = s
# dopóki kolejka nie jest pusta, czyli są jeszcze jakieś wierzchołki do odwiedzenia
while not empty(q):
  # pobierz następny wierzchołek i usuń go z kolejki
  cur_n = front(q)
  pop(q)

  # przerwij jeśli dotarliśmy do celu
  if cur_n == g:
    break
  
  # dla wszystkich krawędzi z aktualnego wierzchołka
  for nh in edges(cur_n):
    # jeśli sąsiad nie był jeszcze odwiedzony
    if nh not in visited:
      # oznacz jako odwiedzony i dodaj do kolejki
      parent[nh] = cur_n
      add(visited, nh)
      push(q, nh)

# ścieżka do wierzchołka docelowego
path = []

# zaczynamy od wierzchołka docelowego i cofamy się po znalezionej ścieżce
cur_n = g
# dopóki nie dotrzemy do startu
while cur_n != None:
  # dodajemy aktualny wierzchołek i przechodzimy do poprzednika
  add(path, cur_n)
  cur_n = parent[cur_n]
# wierzchołki są w odwrotnej kolejności, więc odwracamy listę
reverse(path)

```



# Mapa drogowa

Wyobraźmy sobie teraz problem większej skali, w którym musimy znaleźć najkrótszą drogę pomiędzy miastami. Mając do dyspozycji mapę obszaru, również to zadanie jesteśmy w stanie zapisać w postaci grafu. Jego wierzchołki dla ułatwienia oznaczymy pierwszymi literami nazw miast, choć zamiast tego moglibyśmy użyć również stanów agenta, które w tym wypadku są równe współrzędnym geograficznym. Agent może przemieszczać się jedynie po drogach, które łączą poszczególne miasta, więc miasta pomiędzy którymi istnieją drogi będą połączone krawędziami w grafie. Krawędzie te nie będą miały jednak jednakowej długości i konieczne jest przechowywanie tej informacji jako waga krawędzi. Dla przykładu z rysunku poniżej, aby dotrzeć ze Szczecina do Warszawy, możemy wybrać drogę przez Poznań i Łódź albo drogę przez Gdańsk. W pierwszym przypadku konieczne jest przebycie 3 krawędzi, a w drugim 2, jednakże pierwsza droga będzie krótsza. Z tego powodu, aby znaleźć najkrótsze połączenie nie możemy już wykorzystać algorytmu BFS, ponieważ poszczególne krawędzie (drogi) mają różne długości.
<img src="https://drive.google.com/uc?id=1_3_l4oNKJSSr5QXw-IGUAMFuDUtLIai-"
width="500"/>



# Algorytm Dijkstry

Za pomocą algorytmu Dijkstry możliwe jest znalezienie najkrótszej ścieżki w grafie, którego krawędzie mają różną (ale tylko dodatnią) długość. Wierzchołki znów są odwiedzane w kolejności ich odległości od wierzchołka początkowego, jednakże tym razem konieczne jest wykorzystanie kolejki priorytetowej (kopca), aby to osiągnąć. W każdym wierzchołku sprawdzamy czy da się z niego dojść do jego sąsiadów i ścieżka ta będzie krótsza niż najlepsza dotychczas znaleziona, tzn. czy scieżka, która kończy się aktualnym wierzchołkiem i jego sąsiadem jest aktualnie lepsza. Algorytm ten jest algorytmem zachłannym, ponieważ w każdym swoim kroku dokonuje tylko lokalnie optymalnego wyboru (wybiera wierzchołek, który jest mu najbliżyszy, nie uwzględniając przy tym całej ścieżki).

Zasada działania jest następująca:
* Utwórz dwa zbiory wierzchołków $Q$ i $S$. Początkowo zbiór $Q$ zawiera wszystkie wierzchołki grafu, a zbiór $S$ jest pusty.
* Długość ścieżki wierzchołka początkowego $\text{cost}(s)$ ustaw na 0, a dla pozostałych wierzchołków przyjmij nieskończoność (ponieważ nie znaleziono jeszcze długości najkrótszych ścieżek).
* Ustaw poprzednik $\text{parent}(s)$ wierzchołka startowego na niezdefioniowany. Poprzedniki będą wyznaczały w kierunku odwrotnym najkrótszą ścieżkę od wierzchołka $g$  do wierzchołka startowego $s$.
* W pętli, dopóki zbiór $Q$ zawiera wierzchołek docelowy $g$, wykonaj następujące czynności:
  * Wybierz ze zbioru $Q$ wierzchołek $u$ o najmniejszym koszcie dojścia $\text{cost}(u)$ (w pierwszej iteracji wybierz wierzchołek startowy).
  * Wybrany wierzchołek $u$ usuń ze zbioru $Q$  i dodaj do zbioru $S$.
  * Dla każdego sąsiada $w$  wierzchołka $u$, który jest wciąż w zbiorze $Q$, sprawdź czy:
    *   $\text{cost}(w)$ $>\text{cost}(u)$ +  $\text{weight}(u,w)$.
    *  Jeśli tak, to wyznacz nowy koszt dojścia do wierzchołka w jako:
$\text{cost}(w)$ = $\text{cost}(u)$ +  $\text{weight}(u,w)$. Następnie, wierzchołek $u$ zapisz jako poprzednik $w$: $\text{parent}(w) = u$.

Warto zauważyć, że jeżeli długości krawędzi są równe, algorytm Dijkstry sprowadza się do algorytmu przeszukiwania wszerz (BFS)

```python

# s - wierzchołek startowy
# g - wierzchołek docelowy
# nodes - lista wierzchołków

# zbiór wierzchołków odwiedzonych
visited = set()
# słownik kosztów
cost = {n: inf for n in nodes}
# słownik poprzedników
parent = {s: None}
# utwórz kolejke, w której elementy są ułożone nie w kolejności wprowadzania, lecz w kolejności priorytetu.
q = PriorityQueue()
# dodaj wierzchołek startowy
push(q, (0,s))
cost[s] = 0
# dopóki kolejka nie jest pusta, czyli są jeszcze jakieś wierzchołki do odwiedzenia
while not empty(q):
    # pobierz wierzchołek o najmniejszym priotytecie i usuń go z kolejki
    _, cur_n = top(q)
    pop(q)
    # przerwij jeśli odwiedzony
    if cur_n in visited:
      continue
    # dodaj wierzchołek do listy odwiedonych
    add(visited, cur_n)
      # przerwij jeśli dotarliśmy do celu
    if cur_n == g:
        break
    # dla wszystkich krawędzi z aktualnego wierzchołka    
    for nh, distance in edges(cur_n):
        # przerwij jeśli sąsiad był już odwiedzony
        if nh in visited: 
          continue  
        # pobierz koszt sąsiada lub przypisz mu inf
        old_cost = cost[nh]
        # oblicz koszt dla danego wierzchołka 
        new_cost = cost[cur_n] + distance
        # rozważ nową ścieżkę tylko wtedy, gdy jest lepsza niż dotychczas najlepsze ścieżka
        if new_cost < old_cost:
            # zaktualizuj wartość sąsiada w słowniku kosztów
            cost[nh] = new_cost
            # ustaw poprzednika
            parent[nh] = cur_n
            # dodaj sąsiada do kolejki
            push(q, (new_cost, nh))
# odtwórz ścieżkę
path = []
cur_n = g
while cur_n is not None:
    path.append(cur_n)
    cur_n = parent[cur_n]
reverse(path)
```



#Algorytm A$^*$

Algorytm A* jest bardzo podobny do algorytmu Dijkstry z tą różnicą, że zamiast wybierać do odwiedzin wierzchołek o
najmniejszej możliwej odległości tymczasowej (najbliższy w danym momencie), wybieramy ten, dla którego
minimalną wartość ma funkcja kosztu opisana jako $f(u)=\text{cost}(u)+h(u,g)$. 
Inaczej mówiąc: preferujemy odwiedzanie tych wierzchołków, których suma
przeszacowanej odległości od źródła $\text{cost}(u)$ i niedoszacowanej odległości do celu $h(u,g)$ jest możliwie najmniejsza. W efekcie wybierając kolejne kroki kierujemy się do tych wierzchołków, które według heurystyki są bliżej celu.
Implementacja algorytmu $A^*$ wymaga zdefiniowania dodatkowej funkcji, która będzie odpowiedzialna za obliczanie heurystyki. 
```python
def heuristic(a, b):
   # Manhattan distance on a square grid
   return abs(a.x - b.x) + abs(a.y - b.y)
```
W przedstawionym fragmencie kodu jako heurystykę wykorzystano metrykę Manhattan. Innymi często stosowanymi metrykami są: metryka Euklidesowa czy metryka Czebyszewa. Przyjęcie, że metryka równa się 0 powoduje, że algorytm A* redukuje się do algorytmu Dijkstry. Więcej na temat heurystyk można przeczytać [tutaj](http://theory.stanford.edu/~amitp/GameProgramming/Heuristics.html).
```python
# s - wierzchołek startowy
# g - wierzchołek docelowy
# nodes - lista wierzchołków

# zbiór wierzchołków odwiedzonych
visited = set()
# słownik kosztów
cost = {n: inf for n in nodes}
# słownik poprzedników
parent = {s: None}
# utwórz kolejke, w której elementy są ułożone nie w kolejności wprowadzania, lecz w kolejności priorytetu.
q = PriorityQueue()
# dodaj wierzchołek startowy
push(q, (0,s))
cost[s] = 0
# dopóki kolejka nie jest pusta, czyli są jeszcze jakieś wierzchołki do odwiedzenia
while not empty(q):
    # pobierz wierzchołek o najmniejszym priotytecie i usuń go z kolejki
    _, cur_n = top(q)
    pop(q)
    # przerwij jeśli odwiedzony
    if cur_n in visited:
      continue
    # dodaj wierzchołek do listy odwiedonych
    add(visited, cur_n)
      # przerwij jeśli dotarliśmy do celu
    if cur_n == g:
        break
    # dla wszystkich krawędzi z aktualnego wierzchołka    
    for nh, distance in edges(cur_n):
        # przerwij jeśli sąsiad był już odwiedzony
        if nh in visited: 
          continue  
        # pobierz koszt sąsiada lub przypisz mu inf
        old_cost = cost[nh]
        # oblicz koszt dla danego wierzchołka 
        new_cost = cost[cur_n] + distance
        # rozważ nową ścieżkę tylko wtedy, gdy jest lepsza niż dotychczas najlepsze ścieżka
        if new_cost < old_cost:
            # zaktualizuj wartość sąsiada w słowniku kosztów
            cost[nh] = new_cost
            # ustaw poprzednika
            parent[nh] = cur_n
            # oblicz koszt uzwględniając heurystykę
            priority = new_cost + heuristic(g, nh)
            # dodaj sąsiada do kolejki zgodnie z priorytetem         
            push(q, (priority, nh))
# odtwórz ścieżkę
path = []
cur_n = g
while cur_n is not None:
    path.append(cur_n)
    cur_n = parent[cur_n]
reverse(path)
```


# Uwzględnienie orientacji i bardziej złożone stany

Większość robotów mobilnych potrafi poruszać się tylko w kierunku wzdłużnym, a więc poruszanie się w boki wymaga najpierw skrętu. Taki manewr zajmuje pewien czas, dlatego planując ścieżkę trzeba uwzględnić również koszty wykonania tej operacji, a nie tylko odległość geometryczną od celu.
 
Jeden ze sposobów przekształcenia takiego problemu w graf pokazano na rysunku poniżej. Stan robota możemy teraz opisać wektorem 3-elementowym $(x,y,\theta)$. w którym ostatnia zmienna określa możliwą orientację `['N', 'E', 'S', 'W']`. Jedna komórka siatki jest w grafie reprezentowana przez 4 wierzchołki np. `(1,4,'N'), (1,4,'E'), (1,4,'S'), (1,4,'W')`. Do wierzchołków przypisane są krawędzie oraz odpowiednia waga równa czasowi trwania manewru. Dla tak utworzonego grafu, aby znaleźć najkrótsze połączenie możemy wykorzystać algorytmu Dijkstry albo $A^*$.

<img src="https://drive.google.com/uc?id=1qPcb6jqwAwCgA2tCX4rzPuv3E4ykUYld" width=500/>


# Ciekawe

Porównanie algorytmów w atrakcyjnej wizualnie formie można znaleźć między innymi [tutaj](https://www.redblobgames.com/pathfinding/a-star/introduction.html).

# Przebieg zajęć

Poniżej znajdują się zadania do rozwiązania podczas zajęć. Podczas ich rozwiązywania skorzystaj z PyCharma i szablonów kodów dostępnych w treści zadań. Wprowadzenie do Pythona możesz znaleźć [tutaj](https://colab.research.google.com/drive/1h9ZuXPR-WKHT5giNJdlF6TAhPjRgSS0k#scrollTo=RLRo4F30STIa), natomiast materiały z przedmiotu Technologie Informacyjne znajdują się [tutaj](https://jug.put.poznan.pl/lab-ti/python/).

# Zadanie 1

<img src="https://drive.google.com/uc?id=1YDUVim9-WvMZif6b06PcsKQN_t5RH-NA" width="400" />

Dla grafu z rysunku powyżej znajdź najkrótszą ścieżkę z wierzchołka `1` do wierzchołka `8`:
1. Zdefiniuj graf jako słownik, w którym kluczami będą numery wierzchołów, a wartościami listy ich sąsiadów.
2. Stwórz odpowiednie struktury do przechowywania informacji o odwiedzonych wierzchołkach, aby możliwe było odtworzenie najkrótszej ścieżki.
3. Zaimplementuj algorytm BFS z użyciem `queue.Queue`.
4. Znajdź najkrótszą ścieżkę i wypisz wierzchołki, które ta ścieżka zawiera.

# Zadanie 2

Naszym celem jest znalezienie najkrótszej ścieżki do złota (żółty kwadrat) w świecie 2-D, w którym występują przeszkody w postaci ścian (czarne kwadraty). Agent może poruszać się w 4 kierunkach `['N', 'E', 'S', 'W']`, natomiast jego orientacja jest przez cały czas stała i nie jest istotna w tym zadaniu. 
1. Pobierz [szablon kodu](https://drive.google.com/file/d/1b8OFYNTOTZwQd50kU3B7WFfQywEEyTRn/view?usp=sharing) i otwórz go w PyCharmie.
2. Uruchom testowo program z pliku `main.py`. Powinien się pojawić widok mapy z zaznaczonym agentem (czarna strzałka) oraz złotem.
3. Zastanów się co jest stanem agenta w naszym problemie? Jak można taki stan reprezentować w Pythonie? Za pomocą debuggera/printa sprawdź ile jest możliwych stanów w rozpatrywanym świecie (lista lokacji znajduje się w zmiennej `self.locations`).
4. Jakie są możliwe przejścia pomiędzy stanami (akcje)? Jak obliczyć nowy stan na podstawie poprzedniego i wybranej akcji?
5. Jaką najlepiej wybrać strukturę do oznaczania już odwiedzonych stanów? Co musimy zapisywać, aby później odtworzyć najkrótszą ścieżkę?
6. Zaimplementuj algorytm BFS w funkcji `find_path(self)` z pliku `agent.py` tak, aby zwracała listę lokacji, które należy odwiedzić, aby dotrzeć z `self.loc` do `self.goal`. Nic nie musisz modyfikować w pliku `main.py`.
7. Uzupełnij funkcję `__call__(self)` tak, aby zwracała odpowiednie akcje do wykonania. Pamiętaj o aktualizacji aktualnego stanu agenta i planu. Akcje w niniejszym zadaniu mają postać jednej z wartości `['N', 'E', 'S', 'W']`.


# Zadanie 2 - odpowiedzi

1. 


# Zadanie 3

<img src="https://drive.google.com/uc?id=1_BjHufvHq3PWypdOIRRM2DkW-Kdgj3eQ" width="400" />

Dla grafu z rysunku powyżej znajdź najkrótszą ścieżkę z wierzchołka `1` do wierzchołka `8`:
1. Zdefiniuj graf jako słownik, w którym kluczami będą numery wierzchołów, a wartościami listy ich sąsiadów wraz z długościami krawędzi.
2. Stwórz odpowiednie struktury do przechowywania informacji o odwiedzonych wierzchołkach, aby możliwe było odtworzenie najkrótszej ścieżki.
3. Zaimplementuj algorytm Dijkstry z użyciem `queue.PriorityQueue`.
4. Znajdź najkrótszą ścieżkę i wypisz wierzchołki, które ta ścieżka zawiera.

# Zadanie 4

Rozważmy teraz problem, w którym celem także jest jak najszybsze dotarcie do złota, jednak zamiast poszukiwać go w pomieszczeniach, musimy dotrzeć do miasta, w którym jest ukryte. Do dyspozycji mamy mapę z zaznaczonymi miastami i drogami, których można używać do przemieszczania się pomiędzy nimi. W poprzednim zadaniu przemieszczenie się pomiędzy sąsiadującymi polami zawsze zajmowało tyle samo czasu, ponieważ pola były od siebie równo odległe. Niestety, w tym przypadku drogi mają różną długość i w związku z tym czas ich przebycia jest różny. Czy, w związku z tym, do znalezienia najkrótszej drogi można także użyć algorytmu BFS? Jeśli nie, to jaki algorytm należy użyć? Wykorzystaj [szablon kodu](https://drive.google.com/file/d/1C_qz3XeVhhKRApJIxrtWo2ISDAL-dAuE/view?usp=sharing) i napisz program, który w najkrótszym czasie doprowadzi agenta do złota. Przyjmij, że prędkość poruszania się po wszystkich drogach jest stała i proporcjonalna do długości drogi:
1. Przyjrzyj się jak jest zdefiniowany graf w pliku `main.py`. Co to za struktura? Czym są klucze a czym wartości?
2. Mając dany stan `(x, y)` i wykorzystując strukturę z definicją grafu, jak określić wszystkie lokacje, do którch można się dostać z tego stanu (będzie to równoważne z możliwymi akcjami)? Jak określić długości krawędzi?
3. Co w tym wypadku musimy zapisywać o odwiedzanych stanach?
4. Jaką strukturę najlepiej wybrać do przechowywania kolejki stanów do odwiedzenia?
5. Zaimplementuj odpowiedni algorytm w pliku `agent.py` w funkcji `find_path(self)` tak, aby zwracała ona listę lokacji, które należy odwiedzić, aby dostrzeć z `self.loc` do `self.goal`.
6. Uzupełnij funkcję `__call__(self)` tak, aby zwracała odpowiednie akcje do wykonania. Pamiętaj o aktualizacji aktualnego stanu agenta i planu. Akcje w niniejszym zadaniu moją postać lokacji, do której agent ma się przemieścić. Jeśli lokacji zwróconej jako akcja nie ma na liście sąsiadów aktualnego stanu, agent pozostanie w tym samym miejscu.
7. Czy można jakoś usprawnić przeszukiwanie dużych grafów, aby algorytm najpierw eksplorował kierunki, które są zgodne z azymutem do celu? Zaimplementuj takie usprawnienie i porównaj kolejność odwiedzanych wierzchołków dla przypadku, gdy start jest w lokacji `(9, 4)`, a miejsce docelowe w lokacji `(14, 10)`.

# Zadanie 4 - odpowiedzi

# Zadanie 5

W poprzednich zadaniach szukaliśmy najszybszej ścieżki, ale było to tożsame ze znalezieniem najkrótszej, ponieważ zakładaliśmy stałą prędkość poruszania. Jednakże, za pomocą algorytmów Dijkstry czy A* można także planować ścieżkę, dla której pewien uogólniony koszt (np. czas wykonania operacji czy ilość zużytej energii) jest najmniejszy, a nie tylko długość geometryczna. W tym zadaniu celem jest znalezienie najszybszej ścieżki przy założeniu, że długość wykonywania poszczególnych operacji jest różna. Powracamy do `grid world`, ale tym razem do dyspozycji mamy następujące akcje: `['turnleft', 'turnright', 'forward']`. Orientacja teraz ma znaczenie i akcje `['turnleft', 'turnright']` obracają agenta, odpowiednio, w lewo oraz w prawo. Natomiast akcja `forward` powoduje przesunięcie agenta do przodu w aktualnym kierunku. Czas wykonania poszczególnych operacji to 5 (`turnleft`), 2 (`turnright`) oraz 1 (`forward`). Wykorzystaj [szablon kodu](https://drive.google.com/file/d/1twZ1G0EY6fnQIVIQ0mWjUGA0lnjwJXBP/view?usp=sharing) i napisz program, który pozwoli w jak najkrótszym czasie dotrzeć agentowi do złota:
1. Jakie są teraz możliwe stany agenta? Jak je zapisać?
2. Jak poszczególne akcje zmieniają stan agenta?
3. Uzupełnij funkcję `find_path(self)`, aby zwracała listę stanów, które należy odwiedzić oraz listę akcji, które należy wykonać, aby odwiedzić te stany.
4. Uzupełnij funkcję `__call__(self)`, aby na podstawie obliczonej listy akcji zwracała aktualną akcję do wykonania.